In [49]:
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [50]:
""" pdf_loader = DirectoryLoader('../../training_data/', glob="**/*.pdf")
txt_loader = DirectoryLoader('../../training_data/', glob="**/*.txt")
word_loader = DirectoryLoader('../../training_data/', glob="**/*.docx")
csv_loader = DirectoryLoader('../../training_data/', glob="**/*.csv") """

' pdf_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.pdf")\ntxt_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.txt")\nword_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.docx")\ncsv_loader = DirectoryLoader(\'../../training_data/\', glob="**/*.csv") '

In [51]:
from bs4 import BeautifulSoup as Soup

url = "https://developers.hubspot.com/docs/cms/hubl"
recursive_url_loader = RecursiveUrlLoader(url=url, max_depth=200, extractor=lambda x: Soup(x, "html.parser").text)

loaders = [recursive_url_loader]
documents = []
for loader in loaders:
    documents.extend(loader.load())

print(f"Total number of documents: {len(documents)}")

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
huggingfacehub_api_key= os.getenv('HUGGINGFACEHUB_API_TOKEN')
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings()

Created a chunk of size 11806, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 2934, which is longer than the specified 1000
Created a chunk of size 1552, which is longer than the specified 1000
Created a chunk of size 1250, which is longer than the specified 1000
Created a chunk of size 1770, which is longer than the specified 1000
Created a chunk of size 1263, which is longer than the specified 1000
Created a chunk of size 1404, which is longer than the specified 1000
Created a chunk of size 3872, which is longer than the specified 1000
Created a chunk of size 4866, which is longer than the specified 1000
Created a chunk of size 1139, which is longer than the specified 1000
Created a chunk of size 1211, which is longer than the specified 1000
Created a chunk of size 2934, which is longer than the specified 1000
Created a chunk of 

In [ ]:
db = Chroma.from_documents(texts, embeddings, collection_name="hubspot_hubl_knowledge_base")

In [ ]:
from langchain.chat_models import ChatLiteLLM
from langchain.prompts.chat import (
    ChatPromptTemplate
)

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain.llms import Cohere
cohere_api_key= os.getenv('COHERE_API_KEY')
cohere_llm = ChatLiteLLM(model="command-nightly")


In [ ]:
import os
from dotenv import load_dotenv
from langchain.llms import GooglePalm

load_dotenv()
google_api_key= os.getenv('GOOGLE_API_KEY')
llm = GooglePalm()
llm.temperature = 0.1


In [ ]:
hubl_docs_chain = RetrievalQA.from_chain_type(
    llm=cohere_llm, chain_type="stuff", retriever=db.as_retriever()
)

In [ ]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.utilities import PythonREPL
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import DuckDuckGoSearchRun 
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
)
from langchain.tools import ShellTool
from langchain.agents import load_tools

In [ ]:
#DuckDuckGo Class
duckduck_search = DuckDuckGoSearchRun()
shell_tool = ShellTool()


#Python REPL Class
python_repl = PythonREPL()

shell_tool = Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
)


#DuckDuckGo Tool
duckduck_tool=Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
)

#Python REPL Agent Tool Class
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run
)

In [ ]:
tools = [
    Tool(
        name="HubL Docs",
        func=hubl_docs_chain.run,
        description="Useful for when you need to answer questions about HubL. output should be code",
    ),
    Tool(
        name = "DuckDuckGo",
        func=duckduck_search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
    name="shell",
    description="This tool gives you access to a system's shell. Can be used to run shell commands",
    func=shell_tool.run
    )
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, google_palm_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, handle_parsing_errors=True
)

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor

In [ ]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [ ]:
from langchain.chains import LLMChain

In [ ]:
llm_chain = LLMChain(llm=cohere_llm, prompt=prompt)

In [ ]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
agent_executor.run("Who won the rugby world cup in 2023?")



> Entering new AgentExecutor chain...
 Arrrr, matey! Ye be askin' who won the rugby world cup in 2023? Well, let me tell ye, I can't just magically know answers to every question, I do need some help from ye! Thought: The answer is not something I know by heart, I'll need to look it up. 

Action: DuckDuckGo will help me find the answer.
Action Input: Who won the 2023 rugby world cup? 

Observation:
Observation: DuckDuckGo will help me find the answer. is not a valid tool, try one of [HubL Docs, DuckDuckGo, shell].
Thought:

Retrying langchain.chat_models.litellm.ChatLiteLLM.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: {"message": "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new


 Arrrr, matey! Ye be askin' who won the rugby world cup in 2023? Well shiver me timbers, I can't just tell ye the answer to that one. I need a bit of assistance from ye!

Thought: I can't recall the answer to this one, I guess I will need to use the internet to find out. 

Action: Ya best bet is to use DuckDuckGo to find the answer to this puzzler. 

Action Input: Who won the 2023 rugby world cup? 

Observation:
Observation: Ya best bet is to use DuckDuckGo to find the answer to this puzzler. is not a valid tool, try one of [HubL Docs, DuckDuckGo, shell].
Thought:

Retrying langchain.chat_models.litellm.ChatLiteLLM.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: {"message": "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new




Retrying langchain.chat_models.litellm.ChatLiteLLM.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: {"message": "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new




Retrying langchain.chat_models.litellm.ChatLiteLLM.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: {"message": "You are using a Trial key, which is limited to 5 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.ai/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions"}.



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new




KeyboardInterrupt: 